In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [117]:
#run_time = '28-01-2025_18:28:13'
#run_time = '28-01-2025_18:33:30'
run_time = '02-02-2025_12:58:38'


## Drawing confidence band

In [119]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Open and read the JSON file
path = '../out/run_' + run_time + '/'
with open(path + 'run_info.json', 'r') as file:
    run_info = json.load(file)

# Create a single figure
plt.figure(figsize=(10, 6))

# Iterate over algorithms and plot confidence intervals
for algorithm in run_info['algorithms']:
    algorithm_undersored = algorithm.replace(' ', '_')

    points = pd.read_csv(path + algorithm_undersored + '_points.csv')
    bound = pd.read_csv(path + algorithm_undersored + '_bound.csv')

    # Calculate mean and standard deviation of points for each k
    k_values = points['k'].unique()
    k_values = np.sort(k_values)
    mean_values = np.array([np.sqrt(points[points['k'] == k]['value']).mean() for k in k_values])
    std_values = np.array([np.sqrt(points[points['k'] == k]['value']).std() for k in k_values])
    upper_values = mean_values + 1.96 * std_values
    lower_values = mean_values - 1.96 * std_values

    # Plot confidence interval
    plt.plot(k_values, mean_values, label=algorithm, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), dashes=[5, 5])
    plt.plot(k_values, upper_values, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), alpha = 0.0)
    plt.plot(k_values, lower_values, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), alpha = 0.0)
    plt.fill_between(k_values, lower_values, upper_values, alpha=0.33, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)))

    # Check if bound is all zeros
    if not (bound['value'] == 0).all():
        plt.plot(bound['k'], np.sqrt(bound['value']), label=algorithm + ' bound', linestyle=':', color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)))

# Set title and labels
plt.title('Confidence Intervals for all algorithms')
plt.xlabel('k')
plt.ylabel(r'$\Vert X \Vert_2 / \Vert X_S \Vert_2$')

# Show legend
plt.legend()

# Save plot
plt.tight_layout()
plt.savefig(path + 'confidence_intervals_plot.svg')
plt.close()